In [8]:
import numpy as np
import pandas as pd
from skimage.color import rgb2gray
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import ndimage
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import shutil
from PIL import Image 
import PIL

In [9]:
trainDataFrame = pd.read_json('./data/train/metadata.json').T
trainDataFrame.head()

,label,split,original
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [10]:
len(trainDataFrame)

400

In [14]:
def isDirectoryEmpty(path):
    dir = os.listdir(path)
    if len(dir) == 0:
        return True
    else:
        return False

In [16]:
all_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\all_videos"
fake_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_videos"
real_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\real_videos"

if isDirectoryEmpty(fake_dir_path) and isDirectoryEmpty(real_dir_path):
    for i in range(len(trainDataFrame)):
        if trainDataFrame.iloc[i].label == 'FAKE':
            file_name = trainDataFrame.iloc[i].name

            original = os.path.join(all_dir_path, file_name)
            target   = os.path.join(fake_dir_path, file_name)
            shutil.copyfile(original, target)

        if trainDataFrame.iloc[i].label == 'REAL':
            file_name = trainDataFrame.iloc[i].name

            original = os.path.join(all_dir_path, file_name)
            target   = os.path.join(real_dir_path, file_name)
            shutil.copyfile(original, target)
else:
    print("Directories are not empty")

In [17]:
def saveToFolder(pathToSave, counter, image, x, y, w, h):
    new_path = os.path.join(pathToSave, str(counter) + '.jpg')
    print("Image saved to {}".format(new_path))
    cv2.imwrite(new_path, image[y:y+h, x:x+w])

In [27]:
def face_detection_and_save_image(image, counter, pathToSaveFile, path_to_save_video):
    file_name = 'haarcascade_frontalface_alt.xml'
    lib_dir_path = r"C:\Users\adria\Desktop\hackathon\aplikacja\venv\Lib\site-packages\cv2\data"
    face_cascade_path = original = os.path.join(lib_dir_path, file_name)
    face_cascade = cv2.CascadeClassifier(face_cascade_path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    
    for (x,y,w,h) in faces:
        saveToFolder(pathToSaveFile, counter, image, x, y, w ,h)
#         saveToFolder(path_to_save_video, counter, image, x, y, w ,h)
        image = cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]
        
    return image

In [19]:
def save_faces_from_video(dir_path, parent_path_to_save_video, pathToSaveFile):
    list_files = os.listdir(dir_path)
    counter = 0
    for file in list_files:
        file_directory = file[1:-4]
        print(file)
        print("Cut string: {}".format(file[1:-4]))
        path_to_save_video = os.path.join(parent_path_to_save_video, file_directory)
        os.makedirs(path_to_save_video)
        print("Directory {} created".format(file_directory))
        path = os.path.join(dir_path, file)
        
        vidcap = cv2.VideoCapture(path)
        success, image = vidcap.read()
        while success:
            counter = counter + 1
            print(counter)
            ret, frame = vidcap.read()
            if frame is not None:
                frame = face_detection_and_save_image(frame, counter, pathToSaveFile, path_to_save_video)
            else: 
                break

In [29]:
dir_path = os.path.join(r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_videos")
parent_path_to_save_video = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_in_videos"
pathToSaveFile = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images"
if isDirectoryEmpty(pathToSaveFile) and isDirectoryEmpty(parent_path_to_save_video):
    save_faces_from_video(dir_path, parent_path_to_save_video, pathToSaveFile)
else:
    print("Directories are not empty")

aagfhgtpmv.mp4
Cut string: agfhgtpmv
Directory agfhgtpmv created
1
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\1.jpg
2
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\2.jpg
3
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\3.jpg
4
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\4.jpg
5
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\5.jpg
6
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\6.jpg
7
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\7.jpg
8
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\8.jpg
9
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\9.jpg
10
Image saved to C:\Users\adria\De

Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\81.jpg
82
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\82.jpg
83
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\83.jpg
84
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\84.jpg
85
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\85.jpg
86
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\86.jpg
87
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\87.jpg
88
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\88.jpg
89
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\89.jpg
90
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_fr

Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\161.jpg
162
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\162.jpg
163
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\163.jpg
164
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\164.jpg
165
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\165.jpg
166
167
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\167.jpg
168
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\168.jpg
169
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\169.jpg
170
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\170.jpg
171
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\da

Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\241.jpg
242
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\242.jpg
243
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\243.jpg
244
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\244.jpg
245
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\245.jpg
246
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\246.jpg
247
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\247.jpg
248
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\248.jpg
249
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\249.jpg
250
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\t

500
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\500.jpg
501
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\501.jpg
502
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\502.jpg
503
504
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\504.jpg
505
506
507
508
509
510
511
512
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\512.jpg
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\512.jpg
513
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\513.jpg
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\513.jpg
514
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\514.jpg
515
Image saved to C:\Users\adria\Deskto

...

15508
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15508.jpg
15509
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15509.jpg
15510
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15510.jpg
15511
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15511.jpg
15512
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15512.jpg
15513
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15513.jpg
15514
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15514.jpg
15515
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15515.jpg
15516
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15516.jpg
15517
Image saved to C:\User

Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15639.jpg
15640
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15640.jpg
15641
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15641.jpg
15642
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15642.jpg
15643
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15643.jpg
15644
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15644.jpg
15645
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15645.jpg
15646
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15646.jpg
15647
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15647.jpg
15648
Image saved to C:\Users\adri

Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15721.jpg
15722
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15722.jpg
15723
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15723.jpg
15724
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15724.jpg
15725
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15725.jpg
15726
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15726.jpg
15727
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15727.jpg
15728
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15728.jpg
15729
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15729.jpg
15730
15731
15732
15733
Image save

Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15802.jpg
15803
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15803.jpg
15804
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15804.jpg
15805
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15805.jpg
15806
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15806.jpg
15807
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15807.jpg
15808
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15808.jpg
15809
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15809.jpg
15810
Image saved to C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_from_images\15810.jpg
15811
Image saved to C:\Users\adri

KeyboardInterrupt: 

In [32]:
def create_dataset_to_predict(source_path, dest_path, size_min, size_max):
    list_folder = os.listdir(source_path)
    for folder in list_folder:
        counter = 0
        print(folder)
        fake_faces_folder = os.path.join(source_path,folder)
        print(fake_faces_folder)
        list_images = os.listdir(fake_faces_folder)
        for image in list_images:
            print(counter)
            if counter > size_min and counter < size_max:
                image_path = os.path.join(fake_faces_folder, image)
                
                print("Copy image from:")
                print(image_path)
                print("to:")
                print(dest_path)
                
                
                target   = os.path.join(dest_path,image)
                shutil.copyfile(image_path, target)
                print("Copied")
            counter += 1
        print(counter)

In [33]:
fake_faces = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\fake_faces_in_videos"
new_fake_faces = r"C:\Users\adria\Desktop\hackathon\aplikacja\data_to_train\fake"
if isDirectoryEmpty(new_fake_faces):
    create_dataset_to_predict(source_path=fake_faces, dest_path=new_fake_faces, size_min=0, size_max=50)
else:
    print("Directory is not empty")

Directory is not empty


In [34]:
real_faces = r"C:\Users\adria\Desktop\hackathon\aplikacja\data\train\real_faces_in_videos"
new_real_faces = r"C:\Users\adria\Desktop\hackathon\aplikacja\data_to_train\real"
if isDirectoryEmpty(new_real_faces):
    create_dataset_to_predict(source_path=real_faces, dest_path=new_real_faces, size_min=0, size_max=220)
else:
    print("Directory is not empty")

Directory is not empty


In [35]:
fake_faces = r"C:\Users\adria\Desktop\Praca Magisterska\kaggle-challenge\data\train\fake_faces_in_videos"
new_fake_faces = r"C:\Users\adria\Desktop\Praca Magisterska\kaggle-challenge\data_to_test\fake"
if isDirectoryEmpty(new_fake_faces):
    create_dataset_to_predict(source_path=fake_faces, dest_path=new_fake_faces, size_min=51, size_max=62)
else:
    print("Directory is not empty")

Directory is not empty


In [36]:
DATADIR = r"C:\Users\adria\Desktop\Praca Magisterska\kaggle-challenge\data_to_train"
CATEGORIES = ['real', 'fake']
training_data = []
IMG_SIZE = 100

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        print(class_num)
        print(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass
create_training_data()

0
real
1
fake


In [37]:
import random
random.shuffle(training_data)

In [38]:
X = []
y = []

In [39]:
for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)

In [40]:
print(X.shape)

(26206, 100, 100, 3)


In [41]:
import pickle

pickle_out = open('XColorTrain.pickle', 'wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('yColorTrain.pickle', 'wb')
pickle.dump(y, pickle_out)
pickle_out.close()